In [59]:
import numpy as np
import open3d as o3d
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import laspy
from IPython.display import Image

In [50]:
# data is las
las = laspy.read("data/points2.las")
point_data = np.vstack((las.X, las.Y, las.Z)).transpose()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_data)

In [51]:
o3d.visualization.draw_geometries([pcd])

In [52]:
Image(url="points2.png", width=500, height=300)

In [31]:
"""points = np.asarray(pcd.points)
# Alternatively, you can visualize using matplotlib
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=0.1, c=points[:, 2], cmap="jet")
plt.show()"""

KeyboardInterrupt: 

In [83]:
# KITTI Dataset
# Load binary point cloud
bin_pcd = np.fromfile("data/007480.bin", dtype=np.float32)

# Reshape and drop reflection values
points = bin_pcd.reshape((-1, 4))[:, 0:3]

# Convert to Open3D point cloud
o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))

# Save to whatever format you like
o3d.io.write_point_cloud("data/007480.pcd", o3d_pcd)

True

In [93]:
pcd = o3d.io.read_point_cloud("data/007480.pcd")

In [94]:
print(np.asarray([pcd.points]))

[[[67.82900238 13.75300026  2.54900002]
  [76.86000061 15.84000015  2.86299992]
  [67.38500214 13.99400043  2.53600001]
  ...
  [ 3.79399991 -1.42900002 -1.77499998]
  [ 4.04500008 -1.50999999 -1.89699996]
  [ 3.81500006 -1.40900004 -1.77999997]]]


In [95]:
# CHALLENGE 2 - VOXEL GRID DOWNSAMPLING 

print(f"Points before downsampling: {len(pcd.points)} ") 
# Random down-sampling:
random_pcd = pcd.random_down_sample(sampling_ratio=0.005)

# Uniform down-sampling:
uniform_pcd = pcd.uniform_down_sample(every_k_points=200)

# Voxel down-sampling:
voxel_pcd = pcd.voxel_down_sample(voxel_size=0.1)

print(f"Points after downsampling: {len(random_pcd.points)}")
print(f"Points after downsampling: {len(uniform_pcd.points)}")
print(f"Points after downsampling: {len(voxel_pcd.points)}")

# DOWNSAMPLING 
# o3d.visualization.draw_geometries((downpcd)) 

Points before downsampling: 110736 
Points after downsampling: 553
Points after downsampling: 554
Points after downsampling: 55288


In [96]:
#CHALLENGE 3 - SEGMENTATION

pcd = pcd
_, inliers = pcd.segment_plane(distance_threshold=0.3,ransac_n=3,num_iterations=250)
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1,0,0])
outlier_cloud.paint_uniform_color([0,0,1])
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])


In [97]:
# CHALLENGE 4 - CLUSTERING USING DBSCAN 
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm: 
    labels = np.array(outlier_cloud.cluster_dbscan(eps=0.5, min_points=5, print_progress=True)) 
    
max_label = labels.max() 
print(f"point cloud has {max_label + 1} clusters") 
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1)) 
colors[labels < 0] = 0 
outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3]) 
o3d.visualization.draw_geometries([outlier_cloud]) 

[Open3D DEBUG] Precompute neighbors.
Precompute neighbors.[================>                       ] 37%                                    ] 5%
[Open3D DEBUG] Compute Clusters
Precompute neighbors.[========[Open3D DEBUG] Done Compute Clusters: 466
point cloud has 466 clusters
Precompute neighbors.[========================================] 100%


In [98]:
## BONUS CHALLENGE - CLUSTERING USING KDTREE AND KNN INSTEAD 
pcd_tree = o3d.geometry.KDTreeFlann(outlier_cloud) 
print("Paint the 15th point red.") 
outlier_cloud.colors[15] = [1, 0, 0] 
print("Find its 20 nearest neighbors, paint blue.") 
[k, idx, _] = pcd_tree.search_knn_vector_3d(outlier_cloud.points[15], 20) 
np.asarray(outlier_cloud.colors)[idx[1:], :] = [0, 0, 1] 
o3d.visualization.draw_geometries([outlier_cloud]) 

Paint the 15th point red.
Find its 20 nearest neighbors, paint blue.


In [99]:
# CHALLENGE 5 - BOUNDING BOXES IN 3D
obbs = []
indexes = pd.Series(range(len(labels))).groupby(labels, sort = False).apply(list).tolist()

In [100]:
Max_Points = 1000 
Min_Points = 20 
clusters=[]
for i in range(0, len(indexes)): 
    nb_pts = len(outlier_cloud.select_by_index(indexes[i]).points) 
    if (nb_pts > Min_Points and nb_pts < Max_Points):
        sub_cloud = outlier_cloud.select_by_index(indexes[i]) 
        obb=sub_cloud.get_axis_aligned_bounding_box() 
        obb.color=(0,0,1) 
        obbs.append(obb) 


In [101]:
list_of_visuals = []
list_of_visuals.append(outlier_cloud)
list_of_visuals.extend(obbs)
list_of_visuals.append(inlier_cloud)
o3d.visualization.draw_geometries(list_of_visuals)

In [102]:
Image(url="objectdetection.png", width=500, height=300)